In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df= pd.read_csv("/kaggle/input/appliances-energy-prediction/KAG_energydata_complete.csv", 
                index_col= "date", parse_dates= True, date_format= "%Y-%m-%d %H:%M:%S").reset_index()
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
list(df.columns)

In [ ]:
#restructuring the columns
df.columns= df.columns.str.lower()
df= df.reindex(columns= sorted(df.columns))
df.head()

In [ ]:
df.isnull().sum()

There are no null values in the dataset

In [ ]:
df["weekday"]= np.where(df["date"].dt.weekday>4, 0, 1)
df["weekday"]

In [ ]:
df.head()

In [ ]:
len(df.columns)

In [ ]:
nrows= 5
ncols= int(len(df.columns)/nrows)

fig, axes= plt.subplots(nrows= nrows, ncols= ncols, figsize= (30, 20))
axes= axes.flatten()

# plt.subplots_adjust(hspace=0.05, wspace= 0.0005)


for i, feature in enumerate(df.drop(columns= ["date"]).columns):
    sns.histplot(data= df, x= feature, kde=True, ax= axes[i])
    axes[i].set_title(feature) 

#removing subplots that weren't used
for i in range(len(df.drop(columns= ["date"]).columns), nrows * ncols):
    fig.delaxes(axes[i])
    
fig.suptitle("Distribution of Features", fontsize=18)

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import probplot

nrows= 5
ncols= int(len(df.columns)/nrows)

fig, axes= plt.subplots(nrows= nrows, ncols= ncols, figsize= (30, 27))
axes= axes.flatten()

# plt.subplots_adjust(hspace=0.5)


for i, feature in enumerate(df.drop(columns= ["date"]).columns): 
    probplot(df[feature], dist="norm", plot=axes[i])
    axes[i].set_title(feature)

    
for i in range(len(df.drop(columns= ["date"]).columns), nrows * ncols):
    fig.delaxes(axes[i])

fig.suptitle("Distribution of Numerical Features\n(QQ-Plot)", fontsize=18)
plt.subplots_adjust(top= 2)

plt.tight_layout()
plt.show()

most columns seem to be normally distributed, except for windspeed, rv2, rv1, visibility, t2, rh_6, rh_5 and rh_out

In [ ]:
nrows= 5
ncols= int(len(df.columns)/nrows)

fig, axes= plt.subplots(nrows= nrows, ncols= ncols, figsize= (30, 27))
axes= axes.flatten()

# plt.subplots_adjust(hspace=0.05, wspace= 0.0005)


for i, feature in enumerate(df.drop(columns= ["date"]).columns):
    sns.boxplot(data= df, x= feature, ax= axes[i])
    axes[i].set_title(feature)

    
for i in range(len(df.drop(columns= ["date"]).columns), nrows * ncols):
    fig.delaxes(axes[i])

fig.suptitle("Outlier Detection among Features", fontsize=18)

plt.tight_layout()
plt.show()

features- press_mm_hg, rh_1, rh_2, rh_3, rh_5, rh_7, rh_8, rh_9, rh_out, t1, t2, t2, t4, t5, t6, t7, t8, t_out, tdewpoint, visibility, windspeed all have outliers, which need to be treated before further model development.

In [ ]:
df["lights"].value_counts()

lights feature can be treated as a categorical variable, as the numbers of lights is always one of the 8 given values, also one value (0) significantly dominates the others.

    We can either map the 0 value as 0 and the rest as 1, or perform one hot encoding.
    The first option seems better as 0 is quite dominant with more than 15000 entries out of a total of 19,735.

In [ ]:
df["lights"]= np.where(df["lights"]==0, 0, 1)

In [ ]:
sns.boxplot(data= df, x= "lights")
plt.show()

In [ ]:
#bivariate analysis

nrows= 5
ncols= int(len(df.columns)/nrows)

fig, axes= plt.subplots(nrows= nrows, ncols= ncols, figsize= (30, 10))
axes= axes.flatten()

# plt.subplots_adjust(hspace=0.5)


for i, col in enumerate(df.drop(columns= ["appliances", "date"]).columns):
    
    feature= df[col]
    outcome_var= df["appliances"]
    correlation= feature.corr(outcome_var)
    axes[i].scatter(feature, outcome_var)
    axes[i].set_title(f"Appliances v/s {col} \n correlation: {correlation: .2f}")
    
    #adding a least square line
    z= np.polyfit(feature, outcome_var, deg= 1)
    y_hat= np.poly1d(z)(feature)
    
    axes[i].plot(feature, y_hat, "r--", lw= 1)
    
for i in range(len(df.drop(columns= ["appliances", "date"]).columns), nrows * ncols):
    fig.delaxes(axes[i])


fig.suptitle("Relation b/w outcome variable and features", fontsize=18)

plt.tight_layout()
plt.show()

In [ ]:
#multivariate analysis
#correlation chart

fig= plt.figure(figsize= (20, 20))
sns.heatmap(abs(df.drop(columns= ["appliances", "date"]).corr()), annot= True, fmt= ".2f")
plt.show()

In [ ]:

#dealing with multicollinearity using VIF


from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(df):
    vif= pd.DataFrame()
    vif["variables"]= df.columns
    vif["VIF"]= [variance_inflation_factor(df.values, i)  for i in range(df.shape[1])]
    
    return vif.sort_values(by='VIF',ascending=False).reset_index(drop=True)

In [ ]:
calc_vif(df.drop(columns= ["appliances", "date"]))

In [ ]:
X= df.drop(columns= ["appliances"])
y= df["appliances"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor

In [ ]:
#finding the best ml model

models= {
    "Linear Regression": LinearRegression(),
    "Lasso Regression": Lasso(),
    "Ridge Regression": Ridge(),
    "KNeigghborsRegressor": KNeighborsRegressor(),
    "SVR": SVR(kernel= "rbf"),
    "RandomForestRegressor": RandomForestRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "ExtraTreesRegressor": ExtraTreesRegressor(),
    "XGBRegressor": XGBRegressor()
}

In [ ]:
from time import time

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from scipy import stats

from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Create a list to store explained variances
explained_variances = []

scaler= StandardScaler()

X_scaled = scaler.fit_transform(X_train.drop(columns= ["date"]))

# Define the range of candidate component numbers
n_components_range = range(1, X_scaled.shape[1] + 1)

# Perform PCA for each number of components and store the explained variance
for n_components in n_components_range:

    pca = PCA(n_components=n_components)
    pca.fit(X_scaled)
    explained_variances.append(pca.explained_variance_ratio_.sum())

# Plot the explained variance as a function of the number of components
plt.plot(n_components_range, explained_variances, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.grid(True)
plt.show()

In [ ]:
X_scaled

In [ ]:
meta_data= []

for name, model in models.items():
    model_data= {}
    model_data["name"]= name
    start= time()
    pipeline= Pipeline([
        (
            "Feature Engineering",
            ColumnTransformer([
                ("drop columns", "drop", ["date"])
            ],
                remainder= StandardScaler()
            )
        ),
        (
            "PCA",
            PCA(n_components= 10)
        ),
        (
            "Model",
            model
        )
    ]).fit(X_train, y_train)
    model_data["train time"]= time()-start
    model_data["train r2 score"]= pipeline.score(X_train, y_train)
    model_data["test r2 score"]= r2_score(y_test, pipeline.predict(X_test))
    model_data["rmse score"]= np.sqrt(mean_squared_error(y_test, pipeline.predict(X_test)))
    
    meta_data.append(model_data)

In [ ]:
meta_df= pd.DataFrame(meta_data)
meta_df.sort_values("test r2 score", ascending= False)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

def calculate_lof(X, lof_model):
    lof_scores = -lof_model.fit_predict(X)
    return np.c_[X, lof_scores]

In [ ]:
meta_data= []

for name, model in models.items():
    model_data= {}
    model_data["name"]= name
    start= time()
    pipeline= Pipeline([
        (
            "Feature Engineering",
            ColumnTransformer([
                ("drop columns", "drop", [0]),
                ("scaling", StandardScaler(), slice(1, None)),
                
            ])
        ),
        (
            "outlier treatment",
            FunctionTransformer(
                func=calculate_lof, validate=False,
                kw_args={
                    'lof_model': LocalOutlierFactor(
                        n_neighbors=20, contamination=0.05)
                })
        ),
        (
            "PCA",
            PCA(n_components= 10)
        ),
        (
            "Model",
            model
        )
    ]).fit(X_train, y_train)
    
    model_data["train time"]= time()-start
    model_data["train r2 score"]= pipeline.score(X_train, y_train)
    
    start= time()
    y_pred= pipeline.predict(X_test)
    model_data["predict time"]= time()-start
    
    model_data["test r2 score"]= r2_score(y_test, y_pred)
    model_data["rmse score"]= np.sqrt(mean_squared_error(y_test, y_pred))
    
    meta_data.append(model_data)

In [ ]:
meta_df= pd.DataFrame(meta_data)
meta_df.sort_values("test r2 score", ascending= False)

In [ ]:
def apply_isolation_forest(X):
    isolation_forest = IsolationForest(contamination= "auto")
    labels = isolation_forest.fit_predict(X)
    
    return np.c_[X, labels]

In [ ]:
meta_data= []

for name, model in models.items():
    model_data= {}
    model_data["name"]= name
    
    start= time()  
    pipeline= Pipeline([
        (
            "Feature Engineering",
            ColumnTransformer([
                ("drop columns", "drop", [0]),
                ("scaling", StandardScaler(), slice(1, None)),

            ])
        ),
        (
            "Isolation Forest", 
            FunctionTransformer(func=apply_isolation_forest, validate=False)
        ),
        (
            "PCA",
            PCA(n_components= 10)
        ),
        (
            "Model",
            model
        )
    ]).fit(X_train, y_train)

    model_data["train time"]= time()-start
    
    #training data r2 score
    model_data["train r2 score"]= pipeline.score(X_train, y_train)
    
    start= time()
    y_pred= pipeline.predict(X_test)
    model_data["predict time"]= time()-start
    
    #test data r2 score
    model_data["test r2 score"]= r2_score(y_test, y_pred)
    model_data["rmse score"]= np.sqrt(mean_squared_error(y_test, y_pred))
    
    meta_data.append(model_data)

In [ ]:
meta_df= pd.DataFrame(meta_data)
meta_df.sort_values("test r2 score", ascending= False)

In [ ]:
#we'll choose the ExtraTreesRegressor as it gives the best test r2 score. also, since it is a tree based model it is not affected by 

In [ ]:
preprocessing_pipeline= Pipeline([
    (
        "Feature Engineering",
        ColumnTransformer([
            ("drop columns", "drop", [0]),
            ("scaling", StandardScaler(), slice(1, None))
        ])
    ),
    (
        "outlier treatment",
        FunctionTransformer(
            func=calculate_lof, validate=False,
            kw_args={
                'lof_model': LocalOutlierFactor(
                    n_neighbors=20, contamination=0.05)
            })
    ),
    (
        "PCA",
        PCA(n_components= 10)
    ),
#     (
#         "Model",
#         ExtraTreesRegressor(max_features= None)
#     )
])


In [ ]:
X_train_preprocessed= preprocessing_pipeline.fit_transform(X_train)

In [ ]:
params= {
    "n_estimators": [100, 200, 500],
    "criterion": ["squared_error", "absolute_error"],
    "max_depth": [10, 50, 100, None],
    "max_features": ["sqrt", "log2", None],
    "warm_start": [True, False]
}

In [ ]:
from skopt import BayesSearchCV

bayes_cv= BayesSearchCV(
    ExtraTreesRegressor(max_features= None), 
    search_spaces= params, cv= 3, n_jobs= -1,
    verbose= 1
).fit(X_train_preprocessed, y_train)

In [ ]:
bayes_df= pd.DataFrame(bayes_cv.cv_results_)
bayes_df.sort_values("rank_test_score").head()

In [ ]:
bayes_df["mean_fit_time"].sum()

In [ ]:
bayes_cv.best_params_

In [ ]:
bayes_cv.best_estimator_

In [ ]:
X_test_preprocessed= preprocessing_pipeline.transform(X_test)
y_pred= bayes_cv.predict(X_test_preprocessed)

In [ ]:
def model_analysis(y, y_hat):
    mse= mean_squared_error(y**2, y_hat**2)
    r2= r2_score(y**2, y_hat**2)
    return {"MSE": mse,
            "RMSE": np.sqrt(mse),
            "r2_score(test dataset)": r2}

In [ ]:
model_analysis(y_test, y_pred)

In [ ]:
# final_pipeline= Pipeline([
#     (
#         "preprocessing",
#         preprocessing_pipeline
#     ),
#     (
#         "Model",
#         ExtraTreesRegressor('n_estimators': 500, 'warm_start': True)
#     )
# ])